In [3]:
from datetime import datetime, timedelta
import pytz
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from statistics import stdev
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
pd.options.mode.chained_assignment = None

In [4]:
market = Market()
sec = SEC()
fin = ADatabase("growth")
umod = UniversalModeler()

In [5]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [6]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity']

In [7]:
included_columns = ["year","quarter","ticker","adjclose","y","cash_return_on_equity"]

In [8]:
included_columns.extend(factors)

In [9]:
cfo_col = "netcashprovidedbyusedinoperatingactivities"

In [10]:
training_sets = []
sec.connect()
market.connect()
column_database = {}
for i in tqdm(range(sp5.index.size)):
    try:
        row = sp5.iloc[i]
        ticker = row["Symbol"]
        cik = int(row["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices = prices.groupby(["year","quarter","ticker"]).mean().reset_index()
        financials = sec.retrieve_filing_data(int(cik))
        financials = p.column_date_processing(financials)
        financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in financials["filed"]]
        financials["year"] = [x.year for x in financials["date"]]
        financials["quarter"] = [x.quarter for x in financials["date"]]
        financials = financials[financials["form"]=="10-K"]
        financials["cash_return_on_equity"] = financials[cfo_col] / financials["stockholdersequity"]
        ticker_data = prices.copy()
        ticker_data = ticker_data.merge(financials,on=["year","quarter"],how="left").reset_index()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["cash_return_on_equity"].shift(-4)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(ticker,str(e))
        continue
sec.disconnect()
market.disconnect()

  1%|██                                                                                                                                                                         | 6/490 [00:01<01:55,  4.18it/s]

ACN "['entitycommonstocksharesoutstanding'] not in index"


  2%|██▊                                                                                                                                                                        | 8/490 [00:02<02:17,  3.50it/s]

ADM 'stockholdersequity'


  4%|██████▌                                                                                                                                                                   | 19/490 [00:06<02:39,  2.95it/s]

...

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 474/490 [02:47<00:07,  2.05it/s]

WELL "['propertyplantandequipmentnet'] not in index"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:53<00:00,  1.92it/s]

ZION "['cashandcashequivalentsatcarryingvalue'] not in index"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:53<00:00,  2.82it/s]


In [11]:
data = pd.concat(training_sets)

In [12]:
data = data.dropna().sort_values("year")

In [15]:
data.dropna()

,year,quarter,ticker,adjclose,y,cash_return_on_equity,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,earningspersharebasic,earningspersharediluted,propertyplantandequipmentnet,cashandcashequivalentsatcarryingvalue,entitycommonstocksharesoutstanding,weightedaveragenumberofdilutedsharesoutstanding,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity
48,2012,1,EXC,18.491870,0.336992,0.410292,2.771812e+10,2.829633e+10,8.145000e+08,3.536333e+09,-3.683750e+08,0.953750,1.756364,1.560512e+10,6.609375e+08,3.202880e+08,6.635455e+08,6.619091e+08,6.607700e+09
22,2012,4,TEL,25.232642,0.214095,0.099986,8.277273e+09,7.405800e+09,3.062857e+08,6.400000e+08,3.285000e+08,1.210909,1.200909,3.110400e+09,1.579500e+09,4.225726e+08,4.384545e+08,4.341818e+08,7.722500e+09
3,2012,1,MPC,14.503505,0.315046,0.296771,2.448850e+10,2.448850e+10,6.553333e+08,8.980000e+08,-7.510000e+08,1.651818,1.641818,1.197600e+10,8.582500e+08,3.476141e+08,3.576667e+08,3.560000e+08,8.964250e+09
48,2012,1,D,32.590451,0.251419,0.258500,3.355925e+10,3.355925e+10,8.916667e+08,4.157750e+09,-2.242500e+08,1.511818,1.507273,2.332225e+10,4.562500e+07,2.849429e+08,5.861333e+08,5.851000e+08,9.024500e+09
26,2012,1,AAL,6.984756,-0.189360,-0.217468,2.446800e+10,2.446800e+10,-1.063333e+08,-6.596500e+09,-3.359500e+09,-1.784545,-1.788182,1.437550e+10,1.987500e+08,3.352716e+08,3.206667e+08,3.206667e+08,-4.370000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2022,1,OKE,59.165639,0.395591,0.342791,2.283750e+10,2.335018e+10,3.488063e+08,0.000000e+00,-4.656000e+08,2.623333,2.613333,1.923740e+10,1.759550e+08,4.462133e+08,4.315430e+08,4.303560e+08,6.215764e+09
88,2022,1,TER,126.010322,0.390032,0.436792,3.416262e+09,3.730886e+09,1.071793e+08,5.619900e+08,1.378400e+07,4.536667,4.136667,3.910200e+08,1.018160e+09,1.624170e+08,1.820420e+08,1.671683e+08,1.942994e+09
33,2022,1,HLT,146.812722,-0.866008,-0.634934,1.609800e+10,1.609800e+10,1.023333e+08,-6.527000e+09,-8.195000e+08,0.653333,0.640000,3.255000e+08,2.322500e+09,2.791391e+08,2.826667e+08,2.810000e+08,-1.155500e+09
12,2022,1,DOW,57.008966,0.401920,0.430515,6.218925e+10,6.218925e+10,9.953333e+08,1.814300e+10,-9.982000e+09,2.746667,2.726667,2.051680e+10,3.486333e+09,3.678736e+08,7.446000e+08,7.422000e+08,1.498400e+10


In [14]:
fin.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

  0%|                                                                                                                                                                                     | 0/7 [00:00<?, ?it/s]

Input contains NaN, infinity or a value too large for dtype('float64').


  0%|                                                                                                                                                                                     | 0/7 [00:09<?, ?it/s]

Input contains NaN, infinity or a value too large for dtype('float64').
'str' object has no attribute 'predict'
'str' object has no attribute 'predict'


KeyError: 'cat_prediction'